# 1. Importar Bibliotecas Necessárias
Certifique-se de que todas as bibliotecas necessárias foram importadas.

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tsml.feature_engineering.feature_engineering import FeatureEngineering

import warnings
warnings.filterwarnings("ignore")

# 2. Carregar os Dados
Carregue o dataset e faça o pré-processamento básico.

In [2]:
data_dir = os.path.join('.', './tsml/datasets')
data_file = os.path.join(data_dir, 'day.csv')
data = pd.read_csv(data_file)

Pré-processamento dos dados

In [3]:
# Converter a coluna 'dteday' para datetime
data['dteday'] = pd.to_datetime(data['dteday'])

# Definir colunas categóricas
categorical_cols = ['season', 'weathersit', 'holiday', 'workingday']

# Extrair características da data
data['year'] = data['dteday'].dt.year
data['month'] = data['dteday'].dt.month
data['day'] = data['dteday'].dt.day
data['dayofweek'] = data['dteday'].dt.dayofweek

# Remover a coluna original de data
data.drop('dteday', axis=1, inplace=True)

# Visualizar o dataset pré-processado
data.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,year,month,day,dayofweek
0,1,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985,2011,1,1,5
1,2,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801,2011,1,2,6
2,3,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349,2011,1,3,0
3,4,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562,2011,1,4,1
4,5,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600,2011,1,5,2


In [4]:
# Definir os thresholds de correlação
threshold_min = 0.5
threshold_max = 0.9

# Definir a variável-alvo
target_col = 'cnt'  # Substitua pelo nome da variável-alvo, se necessário

# Calcular a matriz de correlação
correlation_matrix = data.corr()

# Filtrar as colunas com correlação absoluta dentro dos thresholds em relação à variável-alvo
selected_columns = correlation_matrix.index[
    (abs(correlation_matrix[target_col]) >= threshold_min) & (abs(correlation_matrix[target_col]) <= threshold_max)
].tolist()

# Garantir que a variável target_col esteja nos selecionados
if target_col not in selected_columns:
    selected_columns.append(target_col)

# Filtrar o DataFrame original com base nas colunas selecionadas
filtered_data = data[selected_columns+categorical_cols]

# Exibir o resultado
print(f"Colunas selecionadas: {selected_columns+categorical_cols}")

Colunas selecionadas: ['instant', 'yr', 'temp', 'atemp', 'casual', 'year', 'cnt', 'season', 'weathersit', 'holiday', 'workingday']


# 3. Dividir os Dados em Treinamento e Teste
Separe os dados em conjuntos de treinamento e teste.

In [5]:
# Definir as colunas de recurso e a variável alvo
feature_cols = [col for col in data.columns if col != 'cnt']
target_col = 'cnt'

# Dividir em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data[feature_cols], data[target_col],
                                                    test_size=0.9, random_state=42)

# 4. Instanciar a FeatureEngineering
Configure a classe com os parâmetros desejados.

In [6]:
# Instanciar a classe FeatureEngineering
fe = FeatureEngineering(
    apply_boxcox=True,               # Aplicar transformação Box-Cox
    min_poly_degree=1,               # Grau mínimo para polinômios
    max_poly_degree=2,               # Grau máximo para polinômios
    min_comb_size=2,                 # Tamanho mínimo para combinações
    max_comb_size=2,                 # Tamanho máximo para combinações
    lag_features={'temp': [1, 2]},   # Criar defasagens para a coluna 'temp'
    categorical_cols=['season', 'weathersit', 'holiday', 'workingday'],  # Especificar colunas categóricas
    encoding_strategy='auto',        # Estratégia para codificação categórica
    verbose=True                     # Ativar saída detalhada
)

# 5. Ajustar e Transformar os Dados
Aplique as transformações ao conjunto de treinamento e teste.

In [ ]:
# Ajustar e transformar os dados de treinamento
X_train_transformed = fe.fit_transform(X_train, y_train)

# Apenas transformar os dados de teste (sem ajustar os parâmetros novamente)
X_test_transformed = fe.transform(X_test)

# Verificar os dados transformados
X_train_transformed.head()

2024-11-28 09:31:11,719 - WARNING - Skipping Box-Cox for column 'yr' due to non-positive values.
2024-11-28 09:31:11,725 - WARNING - Skipping Box-Cox for column 'holiday' due to non-positive values.
2024-11-28 09:31:11,728 - WARNING - Skipping Box-Cox for column 'weekday' due to non-positive values.
2024-11-28 09:31:11,729 - WARNING - Skipping Box-Cox for column 'workingday' due to non-positive values.
2024-11-28 09:31:11,783 - WARNING - Skipping Box-Cox for column 'dayofweek' due to non-positive values.
2024-11-28 09:31:11,785 - INFO - Step 1/12: Handle missing values... (Elapsed time: 0.00 seconds)
2024-11-28 09:31:11,787 - INFO - Step 2/12: Categorical transformations... (Elapsed time: 0.00 seconds)
2024-11-28 09:31:11,803 - INFO - Step 3/12: Drop low variance features... (Elapsed time: 0.02 seconds)
2024-11-28 09:31:11,809 - INFO - Step 4/12: Handle outliers... (Elapsed time: 0.02 seconds)
2024-11-28 09:31:11,835 - INFO - Step 5/12: Box-Cox transformations... (Elapsed time: 0.05 se

# 6. Treinar um Modelo
Treine um modelo simples para avaliar o impacto da engenharia de recursos.

In [ ]:
# Treinar um modelo de regressão (Random Forest)
model = RandomForestRegressor(random_state=42)
model.fit(X_train_transformed, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test_transformed)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.2f}")

# 7. Salvar os Transformadores e o Modelo
Salve os transformadores e o modelo para uso futuro.

In [ ]:
# Salvar transformadores
fe.save_transformers_dir = './transformers'
fe.save_transformers(X_train_transformed)

# Salvar o modelo
import joblib
joblib.dump(model, './model_rf.pkl')
print("Transformadores e modelo salvos com sucesso.")